## MULTI-LABEL IMAGE CLASSIFICATION WITH KERAS

### LOADING LIBRARIES

In [ ]:
!pip install pillow
!pip install sklearn

In [ ]:

import PIL
import PIL.Image
from sklearn.utils import shuffle
import pandas as pd

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import numpy as np
#from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

In [3]:

import glob,matplotlib as plt
%matplotlib inline

In [ ]:
#from keras.applications.vgg16 import VGG16, preprocess_input,decode_predictions

In [4]:
from keras.applications.densenet import DenseNet201,preprocess_input, decode_predictions

### LOADING DATA

In [5]:
train=pd.read_csv('./meta-data/train.csv')
test=pd.read_csv('./meta-data/test.csv')


In [6]:
img_width, img_height = 224, 224

In [7]:
train.shape,test.shape

((12600, 86), (5400, 1))

In [8]:

import cv2
def read_image(img_path,H,W):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (W,H)) 
    return img


In [9]:
def save_model(model_final,filename):
    # serialize model to JSON
    from keras.models import model_from_json
    model_json = model_final.to_json()
    with open(filename+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model_final.save_weights(filename+".h5")
    print("Saved model to disk")
    
from keras.models import model_from_json
# load json and create model
def load_model(filename):
    json_file = open(filename+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model_final = model_from_json(loaded_model_json)
    # load weights into new model
    model_final.load_weights(filename+".h5")
    print("Loaded model from disk")
    return model_final

In [10]:
import gc
gc.collect()

11

In [11]:
%%time
image_lr=[]
train_img=[]
for i in list(train.Image_name):
    image_lr.append(read_image('./train_img/'+str(i),img_width, img_height))
train_img=np.array(image_lr,dtype=np.float32)
train_img=preprocess_input(train_img,data_format='channels_last')

CPU times: user 3min 35s, sys: 3.8 s, total: 3min 39s
Wall time: 3min 39s


In [12]:
train_img.shape#,test_img.shape

(12600, 224, 224, 3)

In [13]:

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
#rescale = 1./255,   
horizontal_flip = True,
zoom_range = 0.2,
fill_mode="nearest" ,   
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

'''
test_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)
'''
gc.collect()


0

In [14]:
Y=train.iloc[:,1:]
Y.shape

(12600, 85)

In [ ]:
'''
l=int(train.shape[0]-(0.1*train.shape[0]))
l
x_train=train_img[:l,:,:,:]
y_train=Y[:l]
x_valid=train_img[l:,:,:,:]
y_valid=Y[l:]
x_train.shape,y_train.shape,x_valid.shape,y_valid.shape
'''

In [15]:
batch_size=256

In [16]:


train_generator = train_datagen.flow(train_img,Y,
                                    batch_size = batch_size,
                                    shuffle=True)
                                    #class_mode = "categorical")
    
'''
validation_generator = test_datagen.flow(x_valid,y_valid,
                                         batch_size = batch_size,
                                        shuffle=True)
                                        #class_mode = "categorical")

'''

'\nvalidation_generator = test_datagen.flow(x_valid,y_valid,\n                                         batch_size = batch_size,\n                                        shuffle=True)\n                                        #class_mode = "categorical")\n\n'

In [17]:
model =  DenseNet201(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

76398592/76391848 [==============================] - 2s 0us/step


In [18]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False

In [19]:


#Adding custom Layers 
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(85, activation="sigmoid")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


In [20]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 1

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [21]:
adam = Adam(lr=1e-2)
model_final.compile(optimizer=adam, loss='binary_crossentropy', metrics=[fbeta])

In [22]:
#validation_data=validation_generator, validation_steps=x_valid.shape[0]//batch_size, 
model_final.fit_generator(train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=5,shuffle=True)

Epoch 1/5
49/49 [==============================] - 154s 3s/step - loss: 0.3429 - fbeta: 0.7788
Epoch 2/5
49/49 [==============================] - 149s 3s/step - loss: 0.1739 - fbeta: 0.8994
Epoch 3/5
49/49 [==============================] - 149s 3s/step - loss: 0.1418 - fbeta: 0.9194
Epoch 4/5
49/49 [==============================] - 149s 3s/step - loss: 0.1231 - fbeta: 0.9315
Epoch 5/5
49/49 [==============================] - 147s 3s/step - loss: 0.1155 - fbeta: 0.9363


In [24]:
# Train the model 
K.set_value(adam.lr,1e-3)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)
#model_final.fit(train_img, Y,batch_size=256,epochs=3,shuffle=True,verbose=1)

Epoch 1/2
49/49 [==============================] - 152s 3s/step - loss: 0.1002 - fbeta: 0.9464
Epoch 2/2
49/49 [==============================] - 149s 3s/step - loss: 0.0945 - fbeta: 0.9492


In [26]:
K.set_value(adam.lr,1e-3)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 150s 3s/step - loss: 0.0924 - fbeta: 0.9501
Epoch 2/2
49/49 [==============================] - 152s 3s/step - loss: 0.0872 - fbeta: 0.9529


In [29]:
'''
for i,l in enumerate(model_final.layers):
    print(i,l)

'''

'\nfor i,l in enumerate(model_final.layers):\n    print(i,l)\n\n'

In [30]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model_final.layers[:686]:
    layer.trainable = False
for layer in model_final.layers[686:]:
    layer.trainable = True

In [31]:
adam = Adam(lr=1e-3)
model_final.compile(optimizer=adam, loss='binary_crossentropy', metrics=[fbeta])
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=3,
                          shuffle=True)

Epoch 1/3
49/49 [==============================] - 150s 3s/step - loss: 0.0899 - fbeta: 0.9517
Epoch 2/3
49/49 [==============================] - 150s 3s/step - loss: 0.0832 - fbeta: 0.9553
Epoch 3/3
49/49 [==============================] - 149s 3s/step - loss: 0.0779 - fbeta: 0.9578


In [33]:
K.set_value(adam.lr,1e-3)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=5,
                          shuffle=True)

Epoch 1/5
49/49 [==============================] - 150s 3s/step - loss: 0.0729 - fbeta: 0.9613
Epoch 2/5
49/49 [==============================] - 147s 3s/step - loss: 0.0737 - fbeta: 0.9605
Epoch 3/5
49/49 [==============================] - 148s 3s/step - loss: 0.0680 - fbeta: 0.9633
Epoch 4/5
49/49 [==============================] - 151s 3s/step - loss: 0.0650 - fbeta: 0.9657
Epoch 5/5
49/49 [==============================] - 150s 3s/step - loss: 0.0640 - fbeta: 0.9661


In [37]:
K.set_value(adam.lr,1e-4)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=5,
                          shuffle=True)

Epoch 1/3
49/49 [==============================] - 155s 3s/step - loss: 0.0548 - fbeta: 0.9712
Epoch 2/3
49/49 [==============================] - 148s 3s/step - loss: 0.0529 - fbeta: 0.9716
Epoch 3/3
49/49 [==============================] - 151s 3s/step - loss: 0.0527 - fbeta: 0.9721


In [41]:
save_model(model_final,'dl3-densenet')

Saved model to disk


### PREDICTIONG / SUBMISSION

##### LOAD MODEL AND WEIGHTS

In [42]:
gc.collect()

0

In [43]:
%%time
image_lr=[]
test_img=[]
for i in list(test.Image_name):
    image_lr.append(read_image('./test_img/'+str(i),224,224))
test_img=np.array(image_lr,dtype=np.float32)
test_img=preprocess_input(test_img,data_format='channels_last')

CPU times: user 1min 32s, sys: 1.3 s, total: 1min 33s
Wall time: 1min 33s


In [ ]:
#model_final=load_model('./weights/dl3-1')
#batch_size=256

In [ ]:
'''
test_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)
'''

In [56]:
#test_generator = test_datagen.flow(test_img,batch_size = batch_size,shuffle=True)                                      
#pred=model_final.predict_generator(test_generator,verbose=1)
pred=model_final.predict(test_img,batch_size,verbose=1)


5400/5400 [==============================] - 84s 15ms/step


In [57]:
test.shape,test_img.shape

((5400, 1), (5400, 224, 224, 3))

In [58]:
pred.shape

(5400, 85)

In [59]:
sam=pd.read_csv('./meta-data/sample_submission.csv')
sam['Image_name']=test.Image_name
sam.iloc[:,1:]=pred
sam.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,1.094444e-07,1.591067e-12,1.244173e-12,1.525196e-07,1.093585e-10,0.999998,6.114027e-07,3.807507e-12,9.999999e-01,...,2.360347e-07,1.346426e-13,1.894870e-08,1.000000,9.999979e-01,1.195337e-09,1.284207e-22,1.000000,2.283399e-14,0.999995
1,Image-2.jpg,2.754761e-02,2.246699e-02,2.175536e-07,9.509719e-01,1.488121e-03,0.997597,9.969919e-01,1.109599e-04,7.082193e-02,...,3.171766e-03,9.986461e-01,3.669546e-04,0.996574,3.246138e-02,3.058700e-06,1.719193e-05,0.031845,9.989973e-01,0.998435
2,Image-3.jpg,3.537128e-04,7.046590e-10,1.081928e-08,9.999542e-01,3.449667e-11,0.000033,9.999998e-01,1.788751e-04,1.471473e-07,...,9.999621e-01,2.054858e-04,1.241960e-04,0.999860,1.868941e-08,1.285113e-08,3.560143e-14,0.999999,1.390697e-08,0.999999
3,Image-4.jpg,9.999188e-01,8.380170e-05,9.719678e-04,8.691250e-02,9.976994e-01,0.975417,9.731169e-01,2.577786e-07,1.858254e-04,...,9.577435e-02,9.997635e-01,9.991676e-01,0.997518,1.132922e-05,3.245354e-04,1.597107e-06,0.008345,3.486144e-04,0.928228
4,Image-5.jpg,4.883778e-10,2.134624e-13,5.322069e-10,9.990891e-01,1.750810e-10,0.999513,1.715008e-02,3.167303e-14,1.146763e-08,...,2.373168e-06,7.538576e-14,3.353618e-10,0.999999,9.570119e-01,2.828269e-12,2.637620e-19,0.999994,6.505887e-11,0.016051


In [ ]:
#sam=pd.read_csv('sub-densenet-prob.csv')

In [60]:
temp=sam.copy()
for i in sam.iloc[:,1:]:
    temp[i]=sam[i].apply(lambda x: 1 if x>0.5 else 0)
        

In [68]:
temp.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0,0,0,0,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,1
1,Image-2.jpg,0,0,0,1,0,1,1,0,0,...,0,1,0,1,0,0,0,0,1,1
2,Image-3.jpg,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,1
3,Image-4.jpg,1,0,0,0,1,1,1,0,0,...,0,1,1,1,0,0,0,0,0,1
4,Image-5.jpg,0,0,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0


In [64]:
sam.to_csv('sub-densenet-prob.csv',index=False)

In [66]:

temp.to_csv('sub-densenet.csv',index=False)
temp.shape

(5400, 86)